In [83]:
import pandas as pd
import pyodbc
import numpy as np


In [84]:
#Importing tables
#----------------

In [85]:
conn = pyodbc.connect('DSN=COLLEGE;Trusted_connection=yes;')

In [86]:
Students=pd.read_sql("SELECT * FROM students",conn)
Classrooms=pd.read_sql("SELECT * FROM Classrooms",conn)
Teachers=pd.read_sql("SELECT * FROM Teachers",conn)
Departments=pd.read_sql("SELECT * FROM Departments",conn)
Courses=pd.read_sql("SELECT * FROM Courses",conn)

In [87]:
Classrooms.head()

,CourseId,StudentId,degree
0,1,202,92.031070
1,1,12,81.175916
2,1,232,82.396587
3,1,123,62.420449
4,1,274,72.569900


In [88]:
#a. Number of students by Department
#--------------------------------------------

In [91]:
Class_course=pd.DataFrame(data = Classrooms.merge(Courses, on='CourseId',how='inner'))
Students_Departments = pd.merge(Class_course,Departments,how='inner', left_on='DepartmentID', right_on='DepartmentId')
uniq_table = Students_Departments.groupby('DepartmentName').StudentId.nunique()
uniq_table = uniq_table.reset_index()
uniq_table.columns = ['Department Name','Student Count']
uniq_table


,Department Name,Student Count
0,Arts,137
1,English,84
2,Science,200
3,Sport,81


In [92]:
#b. How many studenst has the English teacher by course and in total?
#--------------------------------------------

In [135]:
class_course=pd.DataFrame(data = Classrooms.merge(Courses, on='CourseId',how='inner'))
en_courses=class_course[class_course.DepartmentID ==1].groupby('CourseName').agg({'StudentId':['count']}).reset_index()
en_total_couses=class_course[class_course.DepartmentID ==1].groupby('DepartmentID').StudentId.nunique()
en_courses.columns=['Course name','English students']
df = pd.DataFrame({'Course name': ['---------------','All English students'],'English students': ['-----------',en_total_couses[1]]})
g=pd.concat([en_courses,df])
g

,Course name,English students
0,Advanced English,24
1,English Begginers,32
2,Proffesional English,38
0,---------------,-----------
1,All English students,84


In [94]:
#c. How many small (<22) and bigger (>=22) Classrooms has the Science Department?
#--------------------------------------------

In [136]:
Sc_courses=pd.DataFrame(data=class_course[class_course.DepartmentID ==2].groupby('CourseName').agg({'StudentId':['count']}))
#convert to Arry
for (columnName,columnData) in Sc_courses.iteritems():
    New_Table=columnData.values
    break
Big=0
Small=0
for x in New_Table:
    if int(x)>22:
        Big=Big+1
    else:
        Small=Small+1
print("There are", Big, "big classrooms (more then 22 students) in the Science department")
print("There are", Small, "small classrooms (less then 22 students) in the Science department")

There are 10 big classrooms (more then 22 students) in the Science department
There are 6 small classrooms (less then 22 students) in the Science department


In [137]:
###2d. How many students are by Gender?

In [138]:
Students.groupby('Gender').agg({'StudentId': 'count'}).reset_index()

,Gender,StudentId
0,F,165
1,M,115


In [139]:
##----- 2e. In which courses the percentage of males / females are higher than 70% ?

In [140]:
All_courses=pd.DataFrame(data=Class_course.merge(Students, on='StudentId', how="inner"))
course_gender=pd.DataFrame(data=All_courses.groupby(['CourseId','CourseName','Gender']).agg({'StudentId':['count']})).reset_index()
all_students=pd.DataFrame(data=All_courses.groupby(['CourseId']).agg({'StudentId':['count']}))
Homogenic=pd.DataFrame(data=course_gender.merge(all_students,on ='CourseId',how='inner'))
Homogenic['Gender_ratio']=Homogenic['StudentId_x']/Homogenic['StudentId_y']*100
Homogenic[Homogenic['Gender_ratio']>70].loc[:,['CourseName','Gender_ratio','Gender']]


,CourseName,Gender,Gender_ratio
,,,
42,Sculpture,F,70.833333
52,Tenis,F,72.222222


In [141]:
#######2f. How many students (n and %) have a degree of 80+ by Department?

In [142]:
AllandDep=All_courses.merge(Departments,  left_on='DepartmentID', right_on='DepartmentId', how='inner')
uniq_table=  AllandDep.groupby(['DepartmentID','DepartmentName','StudentId']).agg({'degree':['max']}).reset_index()
best_st=uniq_table[uniq_table['degree','max']>80]
best_std_dep =best_st.groupby(['DepartmentName','DepartmentID']).agg({'StudentId':['count']}).reset_index()
all_st=uniq_table.groupby(['DepartmentID','DepartmentName']).agg({'StudentId':['count']})
all_and_best=best_std_dep.merge(all_st, on='DepartmentID',how='inner')
all_and_best['Ratio']=all_and_best['StudentId_x']/all_and_best['StudentId_y']*100
all_and_best.columns = ['Department','Dep_id','Top students','All students','Over 80 Ratio']
all_and_best.loc[:,['Department', 'Top students', 'All students','Over 80 Ratio']]

,Department,Top students,All students,Over 80 Ratio
0,Arts,61,137,44.525547
1,English,40,84,47.619048
2,Science,122,200,61.000000
3,Sport,39,81,48.148148


In [143]:
##2g. How many students (n and %) have a degree lower than 60 by Department?

In [144]:
AllandDep=All_courses.merge(Departments,  left_on='DepartmentID', right_on='DepartmentId', how='inner')
uniq_table=  AllandDep.groupby(['DepartmentID','DepartmentName','StudentId']).agg({'degree':['min']}).reset_index()
Low_st=uniq_table[uniq_table['degree','min']<60]
Low_std_dep =Low_st.groupby(['DepartmentName','DepartmentID']).agg({'StudentId':['count']}).reset_index()
all_st=uniq_table.groupby(['DepartmentID','DepartmentName']).agg({'StudentId':['count']})
all_and_low=Low_std_dep.merge(all_st, on='DepartmentID',how='inner')
all_and_low['Ratio']=all_and_low['StudentId_x']/all_and_low['StudentId_y']*100
all_and_low.columns = ['Department','Dep_id','Improvment need students','All students','Under 60 Ratio']

all_and_low.loc[:,['Department', 'Improvment need students', 'All students','Under 60 Ratio']]

,Department,Improvment need students,All students,Under 60 Ratio
0,Arts,3,137,2.189781
1,English,2,84,2.380952
2,Science,7,200,3.500000
3,Sport,1,81,1.234568


In [145]:
######2h Rate in descending order the teachers by their student's mean degree.

In [146]:
a=Courses.merge(Teachers, on ='TeacherId' , how='inner')
a['Teacher_name']=a['FirstName']+" "+a['LastName']
b=a.merge(Classrooms, on='CourseId', how='inner')
c=b.groupby(['TeacherId','Teacher_name']).agg({'degree':['mean']}).reset_index()
c.columns=['id','Teacher name','Students mean']
(c.loc[:,['Teacher name','Students mean']]).sort_values(by =['Students mean'],ascending=False)

,Teacher name,Students mean
19,Darick Hess,81.432703
2,Jordyn Baron,80.729494
3,Heather Metcalf,80.440242
17,Zachary Healy,79.365867
0,Kaylea Monzingo,79.365537
15,Daniel Bland,78.908281
21,Keegan Price,78.814631
12,Conor Mckittrick,78.699492
20,Maxwell Romkee,78.673509
16,Terran Beaty,78.495479


In [147]:
##3a. Create a view that shows the courses, departments, teachers and number of students on each

In [148]:
Teachers_courses=((Teachers.merge(Courses, on='TeacherId', how='left'))).merge(Departments,left_on ='DepartmentID',right_on='DepartmentId',how='inner')
C_S=Classrooms.merge(Students,on= 'StudentId',how='inner')
Stu_in_cou=(C_S.groupby(['CourseId']).agg({'StudentId':['count']}).reset_index())
Stu_in_cou.columns=['CourseId','Students']
s=Teachers_courses.merge(Stu_in_cou, on='CourseId',how='left')
s=s.fillna(value=0)
s.loc[:,['DepartmentName','CourseName','FirstName','LastName','Students']]

,DepartmentName,CourseName,FirstName,LastName,Students
0,Science,Computer Science,Kaylea,Monzingo,38.0
1,Science,Mathematics 101,Amanda,Dworkin,18.0
2,Science,Biology,Jordyn,Baron,36.0
3,Science,Statistics,Heather,Metcalf,24.0
4,Science,Data Science,Mackenzie,Olsen,30.0
5,Science,Matematics Advanced,Alexander,Watts,10.0
6,Science,Algebra,Alexander,Watts,12.0
7,Science,Geometry,Alexander,Watts,14.0
8,Science,Physics,Alexander,Watts,12.0
9,Science,Chemistry,Jeffrey,Batzel,24.0


In [149]:
### 3b. Create a view that shows each student, the number of courses taken,their mean degree by department and the total degree mean.

In [150]:
co_dep=Courses.merge(Departments, left_on='DepartmentID', right_on='DepartmentId', how='inner')
cla_st=(Students.merge(Classrooms, on='StudentId',how='left')).merge(co_dep,on='CourseId',how='inner')
d=pd.pivot_table(cla_st,values='degree',index=['StudentId'],columns=['DepartmentName'],aggfunc=np.mean)
e=cla_st.groupby(['StudentId']).agg({'CourseId':['count'],'degree':'mean'})
e.columns=['Course_count','degree_mean']
f=(e.merge(d,on='StudentId',how='inner')).merge(Students, on='StudentId',how='inner')
f.loc[:,['LastName','FirstName','Course_count','Arts','English','Science','Sport','degree_mean']]

,LastName,FirstName,Course_count,Arts,English,Science,Sport,degree_mean
0,Heller,Alexandria,4,86.677535,NaN,79.557508,NaN,81.337515
1,Liska,Cassie,5,79.542738,NaN,78.867115,NaN,79.137364
2,Frank,Hayley,3,NaN,83.972245,76.204427,NaN,78.793700
3,Kotch,Maxwell,3,69.356144,70.929801,74.157066,NaN,71.481003
4,Boehm,Dylan,3,64.494845,NaN,100.000000,65.661106,76.718650
...,...,...,...,...,...,...,...,...
251,Ohler,Elise,4,94.702880,71.617451,80.897496,NaN,82.028830
252,Bannister,Dylan,3,NaN,NaN,76.784735,NaN,76.784735
253,Medina,Bryan,6,78.849330,NaN,81.298837,NaN,80.482335
254,Blas,Sydney,3,NaN,NaN,72.578017,76.749775,75.359189


In [151]:
##ex2-2

In [152]:
co_dep=Courses.merge(Departments, left_on='DepartmentID', right_on='DepartmentId', how='left')
cla_st=(Students.merge(Classrooms, on='StudentId',how='left')).merge(co_dep,on='CourseId',how='left')
d=pd.pivot_table(cla_st,values='degree',index=['StudentId'],columns=['DepartmentName'],aggfunc=np.mean)
e=cla_st.groupby(['StudentId']).agg({'CourseId':['count'],'degree':'mean'})
e.columns=['Course_count','degree_mean']
f=(e.merge(d,on='StudentId',how='inner')).merge(Students, on='StudentId',how='left')
m=h.merge(f,left_on='StudentId',right_on='StudentId',how='left')
m.loc[:,['LastName','FirstName','Course_count','Arts','English','Science','Sport','degree_mean']]

C:\Users\Oren Gadot\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


,LastName,FirstName,Course_count,Arts,English,Science,Sport,degree_mean
0,Heller,Alexandria,4.0,86.677535,NaN,79.557508,NaN,81.337515
1,Liska,Cassie,5.0,79.542738,NaN,78.867115,NaN,79.137364
2,Frank,Hayley,3.0,NaN,83.972245,76.204427,NaN,78.793700
3,Kotch,Maxwell,3.0,69.356144,70.929801,74.157066,NaN,71.481003
4,Boehm,Dylan,3.0,64.494845,NaN,100.000000,65.661106,76.718650
...,...,...,...,...,...,...,...,...
275,Bannister,Dylan,3.0,NaN,NaN,76.784735,NaN,76.784735
276,Medina,Bryan,6.0,78.849330,NaN,81.298837,NaN,80.482335
277,Blas,Sydney,3.0,NaN,NaN,72.578017,76.749775,75.359189
278,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
